In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
!pip install konlpy
from konlpy.tag import Okt
import re
import pickle

from tensorflow import keras
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
# WARNING 무시
warnings.filterwarnings('ignore')

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 45.8 MB/s 


In [3]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [4]:
def clean_sentence(sentence):
    sentence = re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]',r'', sentence)
    return sentence

In [16]:
okt = Okt()
def process_morph(sentence):
    return ' '.join(okt.morphs(sentence))

In [6]:
def clean_and_morph(sentence, is_question=True):
    # 한글 문장 전처리
    sentence = clean_sentence(sentence)
    # 형태소 변환
    sentence = process_morph(sentence)
    # Question 인 경우, Answer인 경우를 분기하여 처리
    if is_question:
        return sentence
    else:
        # <SOS> 토큰은 decoder input에 <EOS> 토큰은 decoder output에 추가
        return ('<SOS> ' + sentence, sentence + ' <EOS>')

In [7]:
def make_prediction(model, question_inputs):
    results = model(inputs=question_inputs, training=False)
    # 변환된 인덱스를 문장으로 변환
    results = np.asarray(results).reshape(-1)
    return results

In [8]:
def make_question(sentence):
    # 코드를 입력하세요
    sentence = clean_and_morph(sentence)
    question_sequence = tokenizer.texts_to_sequences([sentence])
    question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')
    return question_padded

In [10]:
def run_chatbot(question):
    input_seq = make_question(question)
    states = encoder_model2.predict(input_seq)
    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = 2
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model2.predict(
                                                [target_seq] + states)
    # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)

    # 종료 검사
        if index == 3 or len(indexs) >= MAX_LENGTH:
            break


    # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index

        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]
    
    
    results = convert_index_to_text(indexs, tokenizer.word_index['<EOS>'])
    return results

In [11]:
# loading
with open('/content/drive/MyDrive/Colab Notebooks/ssac/nlp/tokenizer/seq2seq_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [12]:
# 모델 파일 로드
encoder_model2 = models.load_model('/content/drive/MyDrive/Colab Notebooks/ssac/nlp/model/encoder_model.h5')
decoder_model2 = models.load_model('/content/drive/MyDrive/Colab Notebooks/ssac/nlp/model/decoder_model.h5')

In [13]:
EMBEDDING_DIM = 100      # 임베딩 벡터 차원
LSTM_HIDDEN_DIM = 128    # LSTM 히든레이어 차원
MAX_LENGTH = 30
TRUNCATING = 'post'      #문장이 30을 넘어갈 경우 뒤에서 자르기
PADDING = 'post'         #뒤에서부터 0으로 패딩
VOCAB_SIZE = len(tokenizer.word_index) + 1  # 나중에 <PAD> 위해 +1

In [14]:
def convert_index_to_text(indexs, end_token): 
    
    sentence = ''
  
    for index in indexs:
        if index == end_token:
            break;
        # 사전에 존재하는 단어의 경우 단어 추가
        if index > 0 and tokenizer.index_word[index] is not None:
            sentence += tokenizer.index_word[index]
        else:
        # 사전에 없는 인덱스면 빈 문자열 추가
            sentence += ''
       
        sentence += ' '
    return sentence

In [18]:
run_chatbot('얼마나 걸리나요?')

'한 십오분 쯤 걸려요 '